# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的船票票號( 'Ticket' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸 :0.780004837244799 (Checked)

df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.780004837244799


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


Name	Sex	Ticket	Cabin	Embarked	Cabin_Count
0	Braund, Mr. Owen Harris	male	A/5 21171	None	S	1014
1	Cumings, Mrs. John Bradley (Florence Briggs Th...	female	PC 17599	C85	C	2
2	Heikkinen, Miss. Laina	female	STON/O2. 3101282	None	S	1014
3	Futrelle, Mrs. Jacques Heath (Lily May Peel)	female	113803	C123	S	2
4	Allen, Mr. William Henry	male	373450	None	S	1014

In [4]:
# 加上 'Cabin' 欄位的計數編碼

cntCbn_df=df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df_Cbn = pd.merge(df, cntCbn_df, on=['Cabin'], how='left')

print(cntCbn_df.shape) # 187, 2  ['Cabin', 'Cabin_Count']
print(df_Cbn.shape) # 1309, 6
print(df_Cbn.columns) #Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Cabin_Count'], dtype='object')
#print(df_Cbn.sort_values(by=['Cabin_Count'], ascending=False))
df_Cbn.head()

(187, 2)
(1309, 6)
Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Cabin_Count'], dtype='object')


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [5]:
# 'Cabin'計數編碼 + 邏輯斯迴歸: 0.7856230275549181 
Le = LabelEncoder()
LR = LogisticRegression()
xx = copy.deepcopy(df_Cbn['Cabin_Count'])

yy=df_Cbn['Cabin'].map(lambda x:hash(x) % 10)

print(xx[0:5])
for i in df_Cbn.columns:
    df_Cbn[i]=Le.fit_transform(df_Cbn[i])
print(xx[0:5])
df_Cbn['Cabin_Count']=xx
cross_val_score(LR, df_Cbn[:train_num],train_Y, cv=5).mean()
#df_Cbn.head(20)

0    1014
1       2
2    1014
3       2
4    1014
Name: Cabin_Count, dtype: int64
0    1014
1       2
2    1014
3       2
4    1014
Name: Cabin_Count, dtype: int64


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

0.7856230275549181

In [6]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸 ==>0.7822709303929694
df_CbH=pd.DataFrame()
for i in df_Cbn.columns:
    df_CbH[i]=Le.fit_transform(df_Cbn[i])
df_CbH['Cabin_Hash']=df_Cbn['Cabin'].map(lambda x:hash(x) % 10)
df_CbH_wo_ccnt=df_CbH.drop(['Cabin_Count'],axis=1)
print(df_CbH_wo_ccnt)
cross_val_score(LR, df_CbH_wo_ccnt[:train_num],train_Y,cv=5).mean()

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


      Name  Sex  Ticket  Cabin  Embarked  Cabin_Hash
0      155    1     720    185         3           5
1      286    0     816    106         0           6
2      523    0     914    185         3           5
3      422    0      65     70         3           0
4       22    1     649    185         3           5
5      818    1     373    185         2           5
6      767    1     109    163         3           3
7      914    1     541    185         3           5
8      605    0     477    185         3           5
9      847    0     174    185         0           5
10    1067    0     839    184         3           4
11     133    0      50     62         3           2
12    1070    1     733    185         3           5
13      38    1     459    185         3           5
14    1221    0     566    185         3           5
15     533    0     204    185         3           5
16     995    1     659    185         2           5
17    1271    1     201    185         3      

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7822709303929694

In [7]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸  0.7833883906834098
df_CbH=pd.DataFrame()

for i in df_Cbn.columns:
    df_CbH[i]=Le.fit_transform(df_Cbn[i])
df_CbH['Cabin_Hash']=df_Cbn['Cabin'].map(lambda x:hash(x) % 10)
df_CbH['Cabin_Count']=xx
print(df_CbH.head())   
cross_val_score(LR, df_CbH[:train_num],train_Y,cv=5).mean()

   Name  Sex  Ticket  Cabin  Embarked  Cabin_Count  Cabin_Hash
0   155    1     720    185         3         1014           5
1   286    0     816    106         0            2           6
2   523    0     914    185         3         1014           5
3   422    0      65     70         3            2           0
4    22    1     649    185         3         1014           5


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

0.7867465521331931

# Answer 
    1st Place # 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸  0.7867465521331931
    3rd Place# 'Cabin'特徵雜湊 + 邏輯斯迴歸 ==>0.7822772783901762
    2nd Place# 'Cabin'計數編碼 + 邏輯斯迴歸: 0.7856230275549181 
    Otehrs# HW 23 mean value encoding : 0.8260501078197707 # 跟 HW 23 不同，HW 對所有 object_features 作均值編碼，今天只對 Cabin 作，所以去除 Survived 後沒有 overfitting 問題
    
    but, it seems, performance is related to lambda x: hash(x) % n   <= this n @5 and @10 got different result different ranking ...



## df_Cbn = pd.merge(df, cntCbn_df, on=['Cabin'], how='left') 這一行 卡了我一小段時間
    原本想說類似 df_Cbn = copy.deepcopy(df), df_Cbn['Cabin_Count'] = cntCbn_df['Cabin_Count'] 
    但是 cntCbn_df is 187 lines (rows)... so... failed  
    df_Cbn[187:]  後全部都是  NaN
    

## Wrong Methon, to combin by df_Cbn['Cabin_Count']=cntCbn_df['Cabin_Count']
#### 加上 'Cabin' 欄位的計數編碼

cntCbn_df=df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()

df_Cbn = copy.deepcopy(df)
df_Cbn['Cabin_Count']=cntCbn_df['Cabin_Count']

print(cntCbn_df.shape) # 187, 2  ['Cabin', 'Cabin_Count']
print(df_Cbn.shape) # 1309, 6
print(df_Cbn.columns) #Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Cabin_Count'], dtype='object')
df_Cbn[187:]

# 這是昨天的練習 ~~

# HW 23 mean value encoding : 0.8260501078197707 
# 跟 HW 23 不同，HW 對所有 object_features 作均值編碼，今天只對 Cabin 作，所以去除 Survived 後沒有 overfitting 問題

df_mean = copy.deepcopy(df[:train_num])
df_mean['Survived']=train_Y
col_mean=df_mean.groupby(['Cabin'])['Survived'].mean().reset_index()
col_mean.columns=['Cabin',f'Cabin_mean']
xx = pd.merge(df_mean, col_mean, on='Cabin', how='left')
print(train_Y[:5])
print(xx.head(100))
xx.shape #(891, 7)

Le = LabelEncoder()
LR = LogisticRegression()
for i in xx.columns:
    xx[i]=Le.fit_transform(xx[i])
xx=xx.drop(['Survived'] , axis=1)
cross_val_score(LR, xx[:train_num],train_Y, cv=10).mean()